This unit is to read the air temperature 
as well as log GPS (bluefly) and humidity to the SD card

We can get all the selected components working and producing 
same file format as the original one.  And then solder together

Then we need to design a 3D-printed housing for it that rigidly 
attaches to an upright profile

Then take it out and run around mapping warm and cold areas 

(Should work on a car ride)

* Fix the humidity work
* Fix the initial logger plot
* Fix the analog reader rate
* missing date records: Rt0000B90Fd"2017-06-24T13:44:54.600"e000F423Fn00000682f00000682o000F423FA


In [302]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [303]:
%sendtofile --source /home/julian/repositories/hangspotdetection/esp32code/hangspotmain.py

Sent 62 lines (1464 bytes) to hangspotmain.py.


In [282]:
%sendtofile --source NewDataLogger_funcs.py

Sent 105 lines (3788 bytes) to NewDataLogger_funcs.py.


In [293]:
%sendtofile --quiet --source BNO055_funcs.py

Sent 81 lines (2729 bytes) to BNO055_funcs.py.


In [11]:
%sendtofile --binary --source ../Sensor_Kennel/OLED_driver.py
%sendtofile --quiet --source BlueFly_funcs.py
%sendtofile --mkdir --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py /lib/
%sendtofile --quiet --source ../Sensor_Kennel/SI7021_funcs.py
%sendtofile --quiet --source ../Sensor_Kennel/OLED_grapher.py
%sendtofile --quiet --source BNO055_funcs.py


Sent 272 lines (8218 bytes) to /lib/sdcard.py.


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'O' is not defined


Sent 55 lines (2040 bytes) to SI7021_funcs.py.
Sent 145 lines (4083 bytes) to OLED_grapher.py.
Sent 112 lines (4423 bytes) to BlueFly_funcs.py.


In [304]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext, oledshowfattext
from machine import Pin, SPI, UART, ADC
import sdcard, os, time, math, ustruct
from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
from NewDataLogger_funcs import ConnectSDcardFile, ConnectBluefly, readlogSI7021, calchumidtemps, readlogbluefly
from BNO055_funcs import InitBNO055, readhexlifyBNO055, GetEulerAngles
from NewDataLogger_funcs import displayEulerAngles

# SD card uses sck=5, mosi=22, miso=27, cs=23
# OLED uses rst=16, scl=15, sda=4
# uartBlueFly = UART(1, baudrate=57600, rx=13, tx=17)
# Pin36 nickel input, Pin19 bridge resistor
# uartBNO055 = UART(2, baudrate=115200, rx=2, tx=18)

# interrupt PRG button used to cycle through modes
butt0 = Pin(0)  
butt0presses = 0
def butt0press(e):
    global butt0presses
    butt0presses += 1
butt0.irq(butt0press, butt0.IRQ_FALLING)

# input value from wire sensor
nickelwire = ADC(Pin(36))
nickelwire.atten(nickelwire.ATTN_11DB)
nickelwire.width(nickelwire.WIDTH_12BIT)
nickelbridge = Pin(19, Pin.OUT)
nickelbridge.value(0)

ConnectBluefly()

if butt0presses == 2:  # just do the blue spot detecting
    import hangspotmain

sdfile = ConnectSDcardFile()
time.sleep_ms(1000) # show file name

SI7021readfail = 0
try:
    i2c.writeto(0x40, b'\xFE')  # resets chip
except OSError:
    SI7021readfail = -1
    oledshowfattext(["SI7021", "bad"])
    time.sleep_ms(700)

k = InitBNO055()
oledshowfattext(["BNO055", k])
time.sleep_ms(1000)

prevflushstamp = 0
prevoledstamp = 0
prevnickelwireread = -100
prevnickelwirestamp = 0
scrollinit()
gpsfixes = 0
prevbutt0 = butt0.value()
prevdisplaymode = 0
prsrec = 100000
while True:
    tstamp = time.ticks_ms()
    displaymode = butt0presses%5
    
    # should use mag = Pin(19, Pin.OUT) to tune it, but doesn't work
    nickelwireread = nickelwire.read()
    if abs(nickelwireread - prevnickelwireread) >= 4 and tstamp > prevnickelwirestamp + 25:
        sdfile.write("Nt{:08X}s{:06X}\n".format(tstamp, nickelwireread))
        prevnickelwireread = nickelwireread
        prevnickelwirestamp = tstamp
        
    if SI7021readfail != -1:
        try:
            readlogSI7021()
        except OSError:
            SI7021readfail += 1
            
    try:
        lbf = readlogbluefly()
    except IndexError:
        lbf = None
    except ValueError:
        lbf = None
    
    bno055rec = readhexlifyBNO055()
    if bno055rec:
        sdfile.write(bno055rec)
        
    if tstamp > prevflushstamp + 2000:
        sdfile.flush()
        prevflushstamp = tstamp

    # all data has been collected.  Now display it
    if type(lbf) == int:
        prsrec = lbf
                
    if displaymode != prevdisplaymode:
        scrollinit()
        k = "display%d"%displaymode
        oledshowfattext(k)
        print(k)
        prevdisplaymode = displaymode
        
    if displaymode == 0:  # display nmea situation
        if type(lbf) == bytes:
            if lbf[0] == ord("V"):
                pass
            elif lbf[0] == ord("Q") and len(lbf) == 43:
                oledshowfattext([lbf[11:19], lbf[29:37], lbf[20:28], lbf[38:42]])
            else:
                oledshowfattext([lbf[:8], lbf[8:16], lbf[16:24], lbf[24:32]])
            #print(lbf)
            
    elif displaymode == 4:
        if bno055rec and tstamp > prevoledstamp + 50:
            displayEulerAngles(*GetEulerAngles())
            bno055rec = None
            prevoledstamp = tstamp

    elif tstamp > prevoledstamp + 200: # scrolling values
        if displaymode == 1:
            addscrollgraph(prsrec-100000, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("prs",0,0,1)
            fbuff.text(str(prsrec),0,8,1)
        elif displaymode == 2:
            addscrollgraph(nickelwireread, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("nickel",0,0,1)
            fbuff.text(str(nickelwireread),0,8,1)
        elif displaymode == 3:
            hs, ts = calchumidtemps()
            addscrollgraph(ts, tstamp)
            plotscrollgraph(fbuff)
            fbuff.text("temp",0,0,1)
            fbuff.text(str(nickelwireread),0,8,1)
        else:
            oledshowfattext(["mstamp", str(tstamp)])
        oledshow()
        prevoledstamp = tstamp


Sent 137 lines (4243 bytes) to main.py.


In [294]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7fd1b096bac8, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [144]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nTraceback (most recent call last):\r\n  File "main.py", line 1, in <module>\r\n  File "OLED_driver.py", line 5, in <module>\r\nKeyboardInterrupt: \r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
